<a href="https://colab.research.google.com/github/miiyu-fujita/MAGIC/blob/main/shapes_GUI_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import / Install necessary libraries

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
! pip install gradio
import gradio as gr

Upload the model
- upload the model zip file included in the github repo to this environment (/content folder), then run this cell to unzip the files

In [ ]:
! unzip "/content/shapes_model-20230303T232527Z-001.zip" -d "/model"

Archive:  /content/shapes_model-20230303T232527Z-001.zip
  inflating: /model/shapes_model/fingerprint.pb  
  inflating: /model/shapes_model/keras_metadata.pb  
  inflating: /model/shapes_model/saved_model.pb  
   creating: /model/shapes_model/assets/
  inflating: /model/shapes_model/variables/variables.index  
  inflating: /model/shapes_model/variables/variables.data-00000-of-00001  


Load the model using tensorflow
- ensure that the path keras is loading the model from matches the path to which you have unzipped the model files

In [ ]:
model = keras.models.load_model('/model/shapes_model')

Build the GUI

In [ ]:
# accept zip files , extract images, present in gallery & predict (2 outputs: gallery of images & predicted zip file)
import zipfile 
import glob
import os
import io
from os import path 
import shutil 

# ----- YOUR IMAGES MUST BE .PNG, OR YOU HAVE TO CHANGE THE GLOB ------
# zip file input -> extract pics to show gallery of uploaded images
def zip_to_gallery(file): # does it work with the file itself? - YESSSS
  print(file.name)
  with zipfile.ZipFile(file.name, 'r') as zip_obj:
    zip_obj.extractall('/input_images')
  return glob.glob('/input_images/**/*.png')

def all_predictions(file): 
  # initialize the parent directory and 3 child directories 
  if not path.exists('/predictions'):
    os.mkdir('/predictions')
    os.mkdir('/predictions/circle')
    os.mkdir('/predictions/square')
    os.mkdir('/predictions/star')
    os.mkdir('/predictions/triangle')
  # get image paths of extracted image files
  image_paths = zip_to_gallery(file) # HARD CODED PATH - USED TO WORK WITH ANY FILE PATH ??W??DSAC><SDKMCSKJDN
  for ipath in image_paths:
    image = tf.keras.utils.load_img(ipath, target_size=(200, 200))
    image_array = tf.keras.utils.img_to_array(image)
    image_array = tf.expand_dims(image_array, 0)
    predictions = model.predict(image_array)
    score = tf.nn.softmax(predictions[0])
    class_names = ['circle', 'square', 'star', 'triangle']

    prediction_label = class_names[np.argmax(score)]
    # here, based on label, save the image to the folder "Igneous", "Metamorphic" or "Sedimentary"
    if prediction_label == 'circle':
      shutil.copy(ipath, '/predictions/circle')
    elif prediction_label == 'square':
      shutil.copy(ipath, '/predictions/square')
    elif prediction_label == 'star':
      shutil.copy(ipath, '/predictions/star')
    else: 
      shutil.copy(ipath, '/predictions/triangle')
  shutil.make_archive('predictions', 'zip', '/', 'predictions')
  return image_paths, "/content/predictions.zip"

def get_circle():
  # assumes main page has an uploaded zip file with predictions 
  # returns a Gallery & zip file 
  rock_1_image_paths = glob.glob('/predictions/circle/*.png')
  shutil.make_archive('circle', 'zip', '/', '/predictions/circle')
  return rock_1_image_paths, "/content/circle.zip"

def get_square():
  rock_2_image_paths = glob.glob('/predictions/square/*.png')
  shutil.make_archive('square', 'zip', '/', '/predictions/square')
  return rock_2_image_paths, "/content/square.zip"

def get_star():
  rock_3_image_paths = glob.glob('/predictions/star/*.png')
  shutil.make_archive('star', 'zip', '/', '/predictions/star')
  return rock_3_image_paths, "/content/star.zip"

def get_triangle():
  rock_4_image_paths = glob.glob('/predictions/triangle/*.png')
  shutil.make_archive('triangle', 'zip', '/', '/predictions/triangle')
  return rock_4_image_paths, "/content/triangle.zip"



all_rock_demo = gr.Interface(all_predictions,
                    gr.File(file_types=[".zip"]),
                    outputs=[gr.Gallery(), "file"],
                    title="MAGIC",
                    description="upload a zip file!")
circle_demo = gr.Interface(get_circle,
                         None,
                         outputs=[gr.Gallery(), "file"],
                         title="MAGIC",
                         description="these are Circles :)")
square_demo = gr.Interface(get_square,
                         None,
                         outputs=[gr.Gallery(), "file"],
                         title="MAGIC",
                         description="these are Squares :)")
star_demo = gr.Interface(get_star,
                         None,
                         outputs=[gr.Gallery(), "file"],
                         title="MAGIC",
                         description="these are Stars :)")
triangle_demo = gr.Interface(get_triangle,
                         None,
                         outputs=[gr.Gallery(), "file"],
                         title="MAGIC",
                         description="these are Triangles :)")



demo = gr.TabbedInterface([all_rock_demo, circle_demo, square_demo, star_demo, triangle_demo], ["All rocks", "Circle", "Square", "Star", "Triangle"])


/usr/local/lib/python3.8/dist-packages/gradio/blocks.py:676: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/blocks.py:676: UserWarning: api_name predict already exists, using predict_2
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/blocks.py:676: UserWarning: api_name predict already exists, using predict_3
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/blocks.py:676: UserWarning: api_name predict already exists, using predict_4
  warnings.warn(


Launch the GUI

In [ ]:
demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

/tmp/circle365c987849249d9dc5c1e5f91d6f0a3c6263735e_wkdp77rf365c987849249d9dc5c1e5f91d6f0a3c6263735e.zip
1/1 [==============================] - 0s 41ms/step
/tmp/TEST_ia2wespyc673c930354926cb8fc53e30d2cf30a8dec51bdb.zip
1/1 [==============================] - 0s 39ms/step
/tmp/TEST__kmi8m2xdc673c930354926cb8fc53e30d2cf30a8dec51bdb.zip
1/1 [==============================] - 0s 42ms/step
/tmp/tester_zip_ab_8qkucc673c930354926cb8fc53e30d2cf30a8dec51bdb.zip
1/1 [==============================] - 0s 58ms/step
/tmp/test_zip_2is0pgys8ad943f2798881f69d93ccc92cd17c5cbe53b378.zip
1/1 [==============================] - 0s 40ms/step
/tmp/tester_zip_zpos0zb1c673c930354926cb8fc53e30d2cf30a8dec51bdb.zip
1/1 [==============================] - 0s 48ms/step
Keyboard interruption in main thread... closing server.
